In [21]:
import pandas as pd
import numpy as np

In [22]:
ec_list = pd.read_csv('ec_list.csv')

# First Excel

In [23]:
replica = pd.read_csv("replica-first_study-03_21_23-trips_dataset.csv", nrows=10000)
replica['dummy_val'] = 1

In [24]:
replica['trip_duration_minutes'] = replica['trip_duration_minutes'].replace(0,1)
replica['trip_duration'] = pd.cut(x=replica['trip_duration_minutes'], bins=[0,5,15,30,60,np.inf], labels=['0-5 mins', '6-15 mins', '16-30 mins','31-60 mins','61+ mins'], ordered =True)

In [25]:
replica['trip_distance'] = pd.cut(x=replica['trip_distance_miles'], bins=[0,1,2,5,10,25,np.inf], labels=['<1 mile','1-2 miles','3-5 miles','6-10 miles','11-25 miles','26+ miles'], ordered =True)

In [26]:
input_dictionaries = {
    'trips_by_primary_mode': {'agg_column':'primary_mode', 'column_header': 'Mode'},
    'trips_by_purpose': {'agg_column': 'trip_purpose', 'column_header': 'Trip Purpose'},
    'trips_by_previous_trip': {'agg_column':'previous_trip_purpose', 'column_header': 'Prev Trip Purpose'},
    'trips_by_vehicle_type': {'agg_column': 'vehicle_type', 'column_header': 'Vehicle Type'},
    'trips_by_origin_land_use': {'agg_column': 'origin_building_use', 'column_header': 'Origin Land Use'}, 
    'trips_by_destination_land_use': {'agg_column': 'destination_land_use', 'column_header': 'Destination Land Use'},
    'trips_by_destination_builidng_use': {'agg_column': 'destination_building_use', 'column_header': 'Destination Building Use'},
    'trips_by_duration': {'agg_column': 'trip_duration', 'column_header': 'Trip Duration'},
    'trips_by_distance': {'agg_column': 'trip_distance', 'column_header': 'Trip Distance'},
    'trips_by_origin_building_use': {'agg_column': 'origin_building_use', 'column_header': 'Origin Building Use'}
}

In [27]:
def number_of_trips_by_manipulation(agregation_column):
    groupby_vals = ['destination_custom', agregation_column] 
    temp_df = replica[groupby_vals + ['dummy_val']]
    grouped = temp_df.groupby(groupby_vals).agg({'count'})
    grouped = grouped.reset_index()
    grouped.columns = ['employment_center', 'agg_column', 'count']
    return grouped

In [28]:
def pivot_table_output(df):
    output = pd.pivot_table(df, values='count', index=['employment_center'], columns=['agg_column'])
    output.columns.name = ''
    return output

In [29]:
def add_header(df, column_header):
    header = pd.MultiIndex.from_product([[column_header], df.columns])
    df.columns = header
    return df

In [30]:
def create_output(key):
    grouped_data = number_of_trips_by_manipulation(agregation_column=input_dictionaries[key]['agg_column'])
    
    pivoted_table = pivot_table_output(grouped_data)
    if key == 'trips_by_previous_trip':
        pivoted_table = pivoted_table.drop('\\N', axis = 1)

    header_added = add_header(df=pivoted_table, column_header=input_dictionaries[key]['column_header'])

    return header_added

In [31]:
output_df = pd.DataFrame()

for input_key in input_dictionaries.keys():
    if output_df.empty:
        output_df = create_output(key=input_key)
        output_df['Total'] = output_df.sum(axis=1)
    else:
        output_df = output_df.merge(create_output(key=input_key), how='inner', left_index=True, right_index=True)

output_df = output_df.merge(ec_list, how='left', left_index=True, right_on='EC_Name')
output_df = output_df.set_index(['EC_ID', 'EC_Name', 'Tier'])
output_df.columns = pd.MultiIndex.from_tuples(output_df.columns)

output_df

c:\Users\cra\Anaconda3\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  return merge(


Mode                                    \
                                    biking carpool commercial on_demand_auto   
EC_ID EC_Name                  Tier                                            
2     Alpine                   4       NaN     1.0        NaN            NaN   
88    Balboa Park              4       NaN     3.0        NaN            NaN   
100   Barona Resort & Casino   6       NaN     1.0        NaN            NaN   
3     Barrio Logan             4       NaN     9.0        2.0            NaN   
4     Carlsbad Palomar Airport 2       2.0    50.0       18.0            NaN   
...                                    ...     ...        ...            ...   
75    Vista Guajome            4       NaN    11.0        NaN            4.0   
76    Vista Sycamore           4       NaN    16.0        2.0            2.0   
77    Vista Tech Park          3       2.0    16.0       18.0            6.0   
78    Vista Village            4       4.0    68.0        7.0            6.0   
79    West Bernardo            2       NaN    10.0        NaN            NaN   

                                                                    \
                                    other_travel_mode private_auto   
EC_ID EC_Name                  Tier                                  
2     Alpine                   4                  2.0          3.0   
88    Balboa Park              4                 22.0          4.0   
100   Barona Resort & Casino   6                  7.0          1.0   
3     Barrio Logan             4                  3.0         13.0   
4     Carlsbad Palomar Airport 2                 36.0        134.0   
...                                               ...          ...   
75    Vista Guajome            4                  1.0         78.0   
76    Vista Sycamore           4                  1.0         69.0   
77    Vista Tech Park          3                  6.0         76.0   
78    Vista Village            4                  9.0        208.0   
79    West Bernardo            2                 30.0          5.0   

                                                            Total  \
                                    public_transit walking          
EC_ID EC_Name                  Tier                                 
2     Alpine                   4               NaN     NaN    6.0   
88    Balboa Park              4               NaN     NaN   29.0   
100   Barona Resort & Casino   6               NaN     NaN    9.0   
3     Barrio Logan             4               NaN     NaN   27.0   
4     Carlsbad Palomar Airport 2               9.0    44.0  293.0   
...                                            ...     ...    ...   
75    Vista Guajome            4               4.0    16.0  114.0   
76    Vista Sycamore           4              13.0    17.0  120.0   
77    Vista Tech Park          3               1.0    26.0  151.0   
78    Vista Village            4              30.0    45.0  377.0   
79    West Bernardo            2               NaN     NaN   45.0   

                                    Trip Purpose  ... Origin Building Use  \
                                      commercial  ...          industrial   
EC_ID EC_Name                  Tier               ...                       
2     Alpine                   4             NaN  ...                 NaN   
88    Balboa Park              4             NaN  ...                 NaN   
100   Barona Resort & Casino   6             NaN  ...                 NaN   
3     Barrio Logan             4             2.0  ...                 2.0   
4     Carlsbad Palomar Airport 2            19.0  ...                18.0   
...                                          ...  ...                 ...   
75    Vista Guajome            4             NaN  ...                 1.0   
76    Vista Sycamore           4             2.0  ...                 5.0   
77    Vista Tech Park          3            19.0  ...                21.0   
78    Vista Village            4             

# Second Excel

In [32]:
replica_2 = pd.read_csv("replica-num_trips_blockgrps_to_ecs_fall_19_thusrday-03_21_23-trips_dataset.csv", nrows=10000)

In [33]:
trips_bgrp_to_EC = replica_2.groupby(['origin_bgrp','destination_custom']).agg({'count'}).reset_index()[['origin_bgrp','destination_custom','origin_cty']].rename(columns={'origin_cty':'trips'})

In [34]:
subset = trips_bgrp_to_EC[trips_bgrp_to_EC['origin_bgrp'].str.contains('San Diego')]
subset.columns = ['origin_bgrp', 'destination_custom', 'trip_count']
output = subset.merge(ec_list, how='left', left_on='destination_custom', right_on='EC_Name')
output['Tract'] = output['origin_bgrp'].str.extract(r'Tract\s(\d+(?:\.\d+)?)')[0]
output['BG'] = output['origin_bgrp'].str.extract(r'(\d+)')
output = output[['EC_ID', 'destination_custom', 'Tract', 'BG', 'trip_count']]
output['Tract'] = output['Tract'].astype(float)
output['BG'] = output['BG'].astype(int)
output['EC_ID'] = output['EC_ID'].astype(int)
output = output.sort_values(['EC_ID', 'Tract']).reset_index(drop=True)
output.columns = ['EC ID', 'Emp Ctr', 'Tract', 'BG', 'Trips']
output

,EC ID,Emp Ctr,Tract,BG,Trips
0,1,San Diego Airport,191.07,1,3
1,1,San Diego Airport,208.01,2,1
2,3,Barrio Logan,191.07,1,1
3,4,Carlsbad Palomar Airport,171.09,2,1
4,4,Carlsbad Palomar Airport,178.11,2,1
...,...,...,...,...,...
1019,107,Valley View Casino & Hotel,191.06,3,18
1020,107,Valley View Casino & Hotel,191.07,1,20
1021,107,Valley View Casino & Hotel,201.03,1,3
1022,107,Valley View Casino & Hotel,207.09,1,1
